In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import warnings
import tensorflow as tf
from Energy_Models import ConvLstm as CL
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('D:\College\Final\Gpr\Datasets\household_power_consumption.txt',sep=';', 
                 parse_dates={'date_time' : ['Date', 'Time']}, infer_datetime_format=True, 
                 low_memory=False, na_values=['nan','?'], index_col='date_time')

In [3]:
df.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
date_time,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2075259 entries, 2006-12-16 17:24:00 to 2010-11-26 21:02:00
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Global_active_power    float64
 1   Global_reactive_power  float64
 2   Voltage                float64
 3   Global_intensity       float64
 4   Sub_metering_1         float64
 5   Sub_metering_2         float64
 6   Sub_metering_3         float64
dtypes: float64(7)
memory usage: 126.7 MB


In [5]:
df.shape

(2075259, 7)

In [6]:
df.isna().sum()

Global_active_power      25979
Global_reactive_power    25979
Voltage                  25979
Global_intensity         25979
Sub_metering_1           25979
Sub_metering_2           25979
Sub_metering_3           25979
dtype: int64

In [7]:
# filling missing values by the value of one day before 
def fill_missing(data):
    one_day = 24*60
    for row in range(data.shape[0]):
        for col in range(data.shape[1]):
            if np.isnan(data[row,col]):
                data[row,col] = data[row-one_day,col]

In [8]:
fill_missing(df.values)

In [9]:
df.isna().sum()

Global_active_power      0
Global_reactive_power    0
Voltage                  0
Global_intensity         0
Sub_metering_1           0
Sub_metering_2           0
Sub_metering_3           0
dtype: int64

In [10]:
df.to_csv('new_household_power_consumption.csv')

In [11]:
df = pd.read_csv('new_household_power_consumption.csv',parse_dates=['date_time'], index_col= 'date_time')

In [12]:
df['sub_metering_remaining'] = (df.Global_active_power * 1000  / 60 ) - (df.Sub_metering_1 + df.Sub_metering_2 + df.Sub_metering_3)
df.describe()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,sub_metering_remaining
count,2.075259e+06,2.075259e+06,2.075259e+06,2.075259e+06,2.075259e+06,2.075259e+06,2.075259e+06,2.075259e+06
mean,1.089418e+00,1.236871e-01,2.408364e+02,4.618401e+00,1.118474e+00,1.291131e+00,6.448635e+00,9.298722e+00
std,1.054678e+00,1.125933e-01,3.240051e+00,4.433165e+00,6.141460e+00,5.796922e+00,8.433584e+00,9.561278e+00
min,7.600000e-02,0.000000e+00,2.232000e+02,2.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00,-2.400000e+00
25%,3.080000e-01,4.800000e-02,2.389900e+02,1.400000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.800000e+00
50%,6.020000e-01,1.000000e-01,2.410000e+02,2.600000e+00,0.000000e+00,0.000000e+00,1.000000e+00,5.500000e+00
75%,1.526000e+00,1.940000e-01,2.428700e+02,6.400000e+00,0.000000e+00,1.000000e+00,1.700000e+01,1.036667e+01
max,1.112200e+01,1.390000e+00,2.541500e+02,4.840000e+01,8.800000e+01,8.000000e+01,3.100000e+01,1.248333e+02


In [13]:
# resample the data on daily basis
df = df.resample('D').mean()
df.shape

(1442, 8)

In [14]:
def train_test_split(df):
    
    # compute split point
    end_idx = df.shape[0]* 70 // 100
    
    train_data = df.iloc[:end_idx, : ]
    test_data = df.iloc[end_idx:, :]
    
    return train_data, test_data

In [406]:
# Split the data into train and test
X_train, X_test = train_test_split(df)

In [407]:
X_test.shape
X_train.shape

(1009, 8)

In [408]:
def scale_data(train, test):
    scaler = MinMaxScaler().fit(train)
    return scaler.transform(train), scaler.transform(test), scaler

In [409]:
X_train, X_test, scaler = scale_data(X_train, X_test)

In [410]:
def convert_to_supervised(df):

    input_features = []
    ouput_feature = []
    
    len_df = df.shape[0]
    
    for i in range(len_df):
        
        end_idx = i + 1 
        
        if end_idx > len_df-1:
            break
            
        input_x , output_y = df[i:end_idx, 1:], df[end_idx: end_idx+1, 0]
        
        input_features.append(input_x)
        ouput_feature.append(output_y)
    
    return np.array(input_features), np.mean(np.array(ouput_feature), axis=1)

In [411]:
# Split the training data into input features and out feature
X_train, Y_train = convert_to_supervised(X_train)
print('Shape of (training data) input features : %s and ouput feature %s' % (X_train.shape, Y_train.shape))

Shape of (training data) input features : (1008, 1, 7) and ouput feature (1008,)


In [412]:
# Split the testing data into input features and out feature
X_test, Y_test = convert_to_supervised(X_test)
print('Shape of (testing data) input features : %s and ouput feature %s' % (X_test.shape, Y_test.shape))

Shape of (testing data) input features : (432, 1, 7) and ouput feature (432,)


In [415]:
n_steps, n_features = X_train.shape[1], X_train.shape[2]
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)
convlstm_model = CL.ConvLstm(n_steps,n_features).getModel()

In [416]:
X_train=X_train.reshape((X_train.shape[0],1,1,n_steps,X_train.shape[2]))
X_test=X_test.reshape((X_test.shape[0],1,1,n_steps,X_test.shape[2]))

In [417]:
convlstm_model.compile(optimizer='adam', loss ='mse')
history = convlstm_model.fit(X_train , Y_train, epochs=200, batch_size=256, verbose= 1,validation_split=0.3)

Epoch 1/200
3/3 [==============================] - 9s 356ms/step - loss: 0.1037 - val_loss: 0.0805
Epoch 2/200
3/3 [==============================] - 0s 74ms/step - loss: 0.0887 - val_loss: 0.0661
Epoch 3/200
3/3 [==============================] - 0s 72ms/step - loss: 0.0738 - val_loss: 0.0522
Epoch 4/200
3/3 [==============================] - 0s 73ms/step - loss: 0.0592 - val_loss: 0.0392
Epoch 5/200
3/3 [==============================] - 0s 73ms/step - loss: 0.0459 - val_loss: 0.0278
Epoch 6/200
3/3 [==============================] - 0s 74ms/step - loss: 0.0342 - val_loss: 0.0193
Epoch 7/200
3/3 [==============================] - 0s 70ms/step - loss: 0.0259 - val_loss: 0.0153
Epoch 8/200
3/3 [==============================] - 0s 73ms/step - loss: 0.0224 - val_loss: 0.0169
Epoch 9/200
3/3 [==============================] - 0s 70ms/step - loss: 0.0239 - val_loss: 0.0202
Epoch 10/200
3/3 [==============================] - 0s 71ms/step - loss: 0.0257 - val_loss: 0.0198
Epoch 11/200
3/3 [

Epoch 84/200
3/3 [==============================] - 0s 69ms/step - loss: 0.0117 - val_loss: 0.0081
Epoch 85/200
3/3 [==============================] - 0s 66ms/step - loss: 0.0117 - val_loss: 0.0083
Epoch 86/200
3/3 [==============================] - 0s 65ms/step - loss: 0.0116 - val_loss: 0.0081
Epoch 87/200
3/3 [==============================] - 0s 68ms/step - loss: 0.0115 - val_loss: 0.0081
Epoch 88/200
3/3 [==============================] - 0s 64ms/step - loss: 0.0117 - val_loss: 0.0081
Epoch 89/200
3/3 [==============================] - 0s 64ms/step - loss: 0.0115 - val_loss: 0.0083
Epoch 90/200
3/3 [==============================] - 0s 64ms/step - loss: 0.0118 - val_loss: 0.0081
Epoch 91/200
3/3 [==============================] - 0s 66ms/step - loss: 0.0117 - val_loss: 0.0080
Epoch 92/200
3/3 [==============================] - 0s 70ms/step - loss: 0.0116 - val_loss: 0.0081
Epoch 93/200
3/3 [==============================] - 0s 65ms/step - loss: 0.0116 - val_loss: 0.0081
Epoch 94/2

3/3 [==============================] - 0s 66ms/step - loss: 0.0114 - val_loss: 0.0081
Epoch 167/200
3/3 [==============================] - 0s 66ms/step - loss: 0.0112 - val_loss: 0.0079
Epoch 168/200
3/3 [==============================] - 0s 65ms/step - loss: 0.0115 - val_loss: 0.0080
Epoch 169/200
3/3 [==============================] - 0s 66ms/step - loss: 0.0112 - val_loss: 0.0082
Epoch 170/200
3/3 [==============================] - 0s 68ms/step - loss: 0.0113 - val_loss: 0.0079
Epoch 171/200
3/3 [==============================] - 0s 65ms/step - loss: 0.0115 - val_loss: 0.0078
Epoch 172/200
3/3 [==============================] - 0s 66ms/step - loss: 0.0112 - val_loss: 0.0081
Epoch 173/200
3/3 [==============================] - 0s 67ms/step - loss: 0.0111 - val_loss: 0.0082
Epoch 174/200
3/3 [==============================] - 0s 72ms/step - loss: 0.0112 - val_loss: 0.0078
Epoch 175/200
3/3 [==============================] - 0s 71ms/step - loss: 0.0112 - val_loss: 0.0078
Epoch 176/200


In [418]:
y_pred_train = convlstm_model.predict(X_train)
y_pred_test  = convlstm_model.predict(X_test) 

14/14 [==============================] - 0s 10ms/step


In [420]:
convlstm_model.evaluate(X_train,Y_train)

32/32 [==============================] - 0s 11ms/step - loss: 0.0101


0.010141022503376007

In [29]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_squared_log_error, mean_absolute_percentage_error

def root_mean_squared_error(y_true, y_pred):    
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [421]:
print('Train RMSE value for LSTM Model  : %.3f ' % root_mean_squared_error(Y_train, y_pred_train))
print('Train MSE value for LSTM Model  : %.3f ' % mean_squared_error(Y_train, y_pred_train))
print('Train R2 value for LSTM Model  : %.3f ' % r2_score(Y_train, y_pred_train))
print('Train MAPE value for LSTM Model  : %.3f ' % mean_absolute_percentage_error(Y_train, y_pred_train))
print('Train RMLSE value for LSTM Model  : %.3f ' % mean_squared_log_error(Y_train, y_pred_train))
print('Train MAE value for LSTM Model  : %.3f ' % mean_absolute_error(Y_train, y_pred_train))
print('---------------------------------------------')
print('Test RMSE value for LSTM Model  : %.3f ' % root_mean_squared_error(Y_test, y_pred_test))
print('Test MSE value for LSTM Model  : %.3f ' % mean_squared_error(Y_test, y_pred_test))
print('Test R2 value for LSTM Model  : %.3f ' % r2_score(Y_test, y_pred_test))
print('Test MAPE value for LSTM Model  : %.3f ' % mean_absolute_percentage_error(Y_test, y_pred_test))
print('Test RMLSE value for LSTM Model  : %.3f ' % mean_squared_log_error(Y_test, y_pred_test))
print('Test MAE value for LSTM Model  : %.3f ' % mean_absolute_error(Y_test, y_pred_test))

Train RMSE value for LSTM Model  : 0.101 
Train MSE value for LSTM Model  : 0.010 
Train R2 value for LSTM Model  : 0.498 
Train MAPE value for LSTM Model  : 146783796208.287 
Train RMLSE value for LSTM Model  : 0.006 
Train MAE value for LSTM Model  : 0.076 
---------------------------------------------
Test RMSE value for LSTM Model  : 0.079 
Test MSE value for LSTM Model  : 0.006 
Test R2 value for LSTM Model  : 0.429 
Test MAPE value for LSTM Model  : 0.257 
Test RMLSE value for LSTM Model  : 0.004 
Test MAE value for LSTM Model  : 0.061 


In [422]:
X_train, X_test = train_test_split(df)

In [423]:
X_train, X_test, scaler = scale_data(X_train, X_test)

In [424]:
def convert_to_supervised(df):

    input_features = []
    ouput_feature = []
    
    len_df = df.shape[0]
    
    for i in range(len_df):
        
        end_idx = i + 7
        
        if end_idx > len_df-7:
            break
            
        input_x , output_y = df[i:end_idx, 1:], df[end_idx: end_idx+7, 0]
        
        input_features.append(input_x)
        ouput_feature.append(output_y)
    
    return np.array(input_features), np.array(ouput_feature)

In [425]:
X_train, Y_train = convert_to_supervised(X_train)
print('Shape of (training data) input features : %s and ouput feature %s' % (X_train.shape, Y_train.shape))

Shape of (training data) input features : (996, 7, 7) and ouput feature (996, 7)


In [426]:
X_test, Y_test = convert_to_supervised(X_test)
print('Shape of (testing data) input features : %s and ouput feature %s' % (X_test.shape, Y_test.shape))

Shape of (testing data) input features : (420, 7, 7) and ouput feature (420, 7)


In [427]:
n_steps, n_features = X_train.shape[1], X_train.shape[2]
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)
convlstm_model = CL.ConvLstm(n_steps,n_features,7).getModel()

In [428]:
X_train=X_train.reshape((X_train.shape[0],1,1,n_steps,X_train.shape[2]))
X_test=X_test.reshape((X_test.shape[0],1,1,n_steps,X_test.shape[2]))

In [429]:
convlstm_model.compile(optimizer='adam', loss ='mae')
history = convlstm_model.fit(X_train , Y_train, epochs=200, batch_size=256, verbose= 1,validation_split=0.3)

Epoch 1/200
3/3 [==============================] - 8s 577ms/step - loss: 0.2882 - val_loss: 0.2734
Epoch 2/200
3/3 [==============================] - 1s 206ms/step - loss: 0.2801 - val_loss: 0.2632
Epoch 3/200
3/3 [==============================] - 1s 241ms/step - loss: 0.2696 - val_loss: 0.2493
Epoch 4/200
3/3 [==============================] - 1s 228ms/step - loss: 0.2557 - val_loss: 0.2310
Epoch 5/200
3/3 [==============================] - 1s 213ms/step - loss: 0.2370 - val_loss: 0.2069
Epoch 6/200
3/3 [==============================] - 1s 208ms/step - loss: 0.2129 - val_loss: 0.1770
Epoch 7/200
3/3 [==============================] - 1s 207ms/step - loss: 0.1834 - val_loss: 0.1474
Epoch 8/200
3/3 [==============================] - 1s 219ms/step - loss: 0.1576 - val_loss: 0.1290
Epoch 9/200
3/3 [==============================] - 1s 208ms/step - loss: 0.1413 - val_loss: 0.1236
Epoch 10/200
3/3 [==============================] - 1s 237ms/step - loss: 0.1340 - val_loss: 0.1142
Epoch 11/

3/3 [==============================] - 1s 225ms/step - loss: 0.0800 - val_loss: 0.0679
Epoch 84/200
3/3 [==============================] - 1s 232ms/step - loss: 0.0797 - val_loss: 0.0682
Epoch 85/200
3/3 [==============================] - 1s 214ms/step - loss: 0.0796 - val_loss: 0.0688
Epoch 86/200
3/3 [==============================] - 1s 225ms/step - loss: 0.0795 - val_loss: 0.0693
Epoch 87/200
3/3 [==============================] - 1s 211ms/step - loss: 0.0795 - val_loss: 0.0685
Epoch 88/200
3/3 [==============================] - 1s 226ms/step - loss: 0.0800 - val_loss: 0.0681
Epoch 89/200
3/3 [==============================] - 1s 223ms/step - loss: 0.0802 - val_loss: 0.0680
Epoch 90/200
3/3 [==============================] - 1s 210ms/step - loss: 0.0798 - val_loss: 0.0686
Epoch 91/200
3/3 [==============================] - 1s 216ms/step - loss: 0.0795 - val_loss: 0.0690
Epoch 92/200
3/3 [==============================] - 1s 211ms/step - loss: 0.0795 - val_loss: 0.0683
Epoch 93/200


In [430]:
y_pred_train = convlstm_model.predict(X_train)
y_pred_test  = convlstm_model.predict(X_test) 

14/14 [==============================] - 0s 14ms/step


In [432]:
print('Train RMSE value for LSTM Model  : %.3f ' % root_mean_squared_error(Y_train, y_pred_train))
print('Train MSE value for LSTM Model  : %.3f ' % mean_squared_error(Y_train, y_pred_train))
print('Train R2 value for LSTM Model  : %.3f ' % r2_score(Y_train, y_pred_train))
print('Train MAPE value for LSTM Model  : %.3f ' % mean_absolute_percentage_error(Y_train, y_pred_train))
#print('Train RMLSE value for LSTM Model  : %.3f ' % mean_squared_log_error(Y_train, y_pred_train))
print('Train MAE value for LSTM Model  : %.3f ' % mean_absolute_error(Y_train, y_pred_train))
print('---------------------------------------------')
print('Test RMSE value for LSTM Model  : %.3f ' % root_mean_squared_error(Y_test, y_pred_test))
print('Test MSE value for LSTM Model  : %.3f ' % mean_squared_error(Y_test, y_pred_test))
print('Test R2 value for LSTM Model  : %.3f ' % r2_score(Y_test, y_pred_test))
print('Test MAPE value for LSTM Model  : %.3f ' % mean_absolute_percentage_error(Y_test, y_pred_test))
#print('Test RMLSE value for LSTM Model  : %.3f ' % mean_squared_log_error(Y_test, y_pred_test))
print('Test MAE value for LSTM Model  : %.3f ' % mean_absolute_error(Y_test, y_pred_test))

Train RMSE value for LSTM Model  : 0.103 
Train MSE value for LSTM Model  : 0.011 
Train R2 value for LSTM Model  : 0.455 
Train MAPE value for LSTM Model  : 29462041405.988 
Train MAE value for LSTM Model  : 0.074 
---------------------------------------------
Test RMSE value for LSTM Model  : 0.081 
Test MSE value for LSTM Model  : 0.007 
Test R2 value for LSTM Model  : 0.408 
Test MAPE value for LSTM Model  : 0.264 
Test MAE value for LSTM Model  : 0.062 


In [433]:
convlstm_model.evaluate(X_train,Y_train)

32/32 [==============================] - 1s 15ms/step - loss: 0.0744


0.07443315535783768

In [434]:
convlstm_model.evaluate(X_test,Y_test)

14/14 [==============================] - 0s 18ms/step - loss: 0.0623


0.062288351356983185